In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import re
from collections import defaultdict, Counter
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import urllib.request
import zipfile

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

Path("../model").mkdir(parents=True, exist_ok=True)
Path("../result").mkdir(parents=True, exist_ok=True)
Path("../data").mkdir(parents=True, exist_ok=True)

Using device: mps


In [12]:
def load_data(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                words = [word for word in line.split() if word.strip()]
                if words:
                    sentences.append(words)
    return sentences

def words_to_chars_labels(sentences):
    char_sequences = []
    label_sequences = []
    
    for words in sentences:
        chars = []
        labels = []
        
        for word in words:
            word = word.strip()
            if len(word) == 1:
                chars.append(word)
                labels.append('S')  # Single
            elif len(word) > 1:
                chars.append(word[0])
                labels.append('B')  # Begin
                for char in word[1:-1]:
                    chars.append(char)
                    labels.append('M')  # Middle
                chars.append(word[-1])
                labels.append('E')  # End
        
        if chars:  # 只添加非空序列
            char_sequences.append(chars)
            label_sequences.append(labels)
    
    return char_sequences, label_sequences

train_sentences = load_data("../data/icwb2-data/training/msr_training.utf8")
test_sentences = load_data("../data/icwb2-data/testing/msr_test.utf8")

train_chars, train_labels = words_to_chars_labels(train_sentences)
test_chars, test_labels = words_to_chars_labels(test_sentences)

print(f"训练集句子数量: {len(train_chars)}")
print(f"测试集句子数量: {len(test_chars)}")
print(f"训练集示例: {train_chars[0]}")
print(f"对应标签: {train_labels[0]}")

训练集句子数量: 86918
测试集句子数量: 3985
训练集示例: ['“', '人', '们', '常', '说', '生', '活', '是', '一', '部', '教', '科', '书', '，', '而', '血', '与', '火', '的', '战', '争', '更', '是', '不', '可', '多', '得', '的', '教', '科', '书', '，', '她', '确', '实', '是', '名', '副', '其', '实', '的', '‘', '我', '的', '大', '学', '’', '。']
对应标签: ['S', 'B', 'E', 'S', 'S', 'B', 'E', 'S', 'S', 'S', 'B', 'M', 'E', 'S', 'S', 'S', 'S', 'S', 'S', 'B', 'E', 'S', 'S', 'B', 'M', 'M', 'E', 'S', 'B', 'M', 'E', 'S', 'S', 'B', 'E', 'S', 'B', 'M', 'M', 'E', 'S', 'S', 'S', 'S', 'B', 'E', 'S', 'S']


In [13]:
class MaximumMatching:
    """最大匹配分词算法"""
    
    def __init__(self):
        self.word_dict = set()
        self.max_len = 0
    
    def build_dict(self, sentences):
        """从训练数据构建词典"""
        for words in sentences:
            for word in words:
                word = word.strip()
                if word:
                    self.word_dict.add(word)
                    self.max_len = max(self.max_len, len(word))
        
        print(f"词典大小: {len(self.word_dict)}")
        print(f"最大词长: {self.max_len}")
    
    def forward_max_matching(self, text):
        """正向最大匹配"""
        result = []
        i = 0
        
        while i < len(text):
            # 尝试最长匹配
            matched = False
            for length in range(min(self.max_len, len(text) - i), 0, -1):
                word = text[i:i+length]
                if word in self.word_dict:
                    result.append(word)
                    i += length
                    matched = True
                    break
            
            if not matched:
                # 如果没有匹配，将单个字符作为词
                result.append(text[i])
                i += 1
        
        return result
    
    def backward_max_matching(self, text):
        """逆向最大匹配"""
        result = []
        i = len(text)
        
        while i > 0:
            matched = False
            for length in range(min(self.max_len, i), 0, -1):
                word = text[i-length:i]
                if word in self.word_dict:
                    result.insert(0, word)
                    i -= length
                    matched = True
                    break
            
            if not matched:
                result.insert(0, text[i-1])
                i -= 1
        
        return result
    
    def bidirectional_matching(self, text):
        """双向最大匹配"""
        forward_result = self.forward_max_matching(text)
        backward_result = self.backward_max_matching(text)
        
        # 选择词数更少的结果，如果词数相同，选择单字词更少的
        if len(forward_result) < len(backward_result):
            return forward_result
        elif len(forward_result) > len(backward_result):
            return backward_result
        else:
            # 词数相同，比较单字词数量
            forward_single = sum(1 for word in forward_result if len(word) == 1)
            backward_single = sum(1 for word in backward_result if len(word) == 1)
            
            if forward_single <= backward_single:
                return forward_result
            else:
                return backward_result

# 训练最大匹配模型
mm_model = MaximumMatching()
mm_model.build_dict(train_sentences)

# 测试最大匹配算法
test_text = "我爱北京天安门"
print(f"测试文本: {test_text}")
print(f"正向最大匹配: {mm_model.forward_max_matching(test_text)}")
print(f"逆向最大匹配: {mm_model.backward_max_matching(test_text)}")
print(f"双向最大匹配: {mm_model.bidirectional_matching(test_text)}")

词典大小: 88119
最大词长: 48
测试文本: 我爱北京天安门
正向最大匹配: ['我', '爱', '北京', '天安门']
逆向最大匹配: ['我', '爱', '北京', '天安门']
双向最大匹配: ['我', '爱', '北京', '天安门']


In [14]:
class ChineseSegmentationDataset(Dataset):
    """中文分词数据集"""
    
    def __init__(self, char_sequences, label_sequences, char2idx, label2idx):
        self.char_sequences = char_sequences
        self.label_sequences = label_sequences
        self.char2idx = char2idx
        self.label2idx = label2idx
    
    def __len__(self):
        return len(self.char_sequences)
    
    def __getitem__(self, idx):
        chars = self.char_sequences[idx]
        labels = self.label_sequences[idx]
        
        # 转换为索引
        char_indices = [self.char2idx.get(char, self.char2idx['<UNK>']) for char in chars]
        label_indices = [self.label2idx[label] for label in labels]
        
        return torch.tensor(char_indices), torch.tensor(label_indices)

def collate_fn(batch):
    """批处理函数"""
    chars, labels = zip(*batch)
    
    # 获取最大长度
    max_len = max(len(seq) for seq in chars)
    
    # 填充序列
    padded_chars = []
    padded_labels = []
    lengths = []
    
    for char_seq, label_seq in zip(chars, labels):
        length = len(char_seq)
        lengths.append(length)
        
        # 填充到最大长度
        padded_char = torch.cat([char_seq, torch.zeros(max_len - length, dtype=torch.long)])
        padded_label = torch.cat([label_seq, torch.zeros(max_len - length, dtype=torch.long)])
        
        padded_chars.append(padded_char)
        padded_labels.append(padded_label)
    
    return torch.stack(padded_chars), torch.stack(padded_labels), torch.tensor(lengths)

class BiLSTMCRF(nn.Module):
    """基于BiLSTM-CRF的中文分词模型"""
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags, dropout=0.1):
        super(BiLSTMCRF, self).__init__()
        
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.num_tags = num_tags
        
        # 字符嵌入层
        self.char_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # BiLSTM层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, 
                           num_layers=2, bidirectional=True, 
                           dropout=dropout, batch_first=True)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # 输出层
        self.hidden2tag = nn.Linear(hidden_dim, num_tags)
        
        # CRF转移矩阵
        self.transitions = nn.Parameter(torch.randn(num_tags, num_tags))
        
        # 初始化参数
        self.init_weights()
    
    def init_weights(self):
        """初始化模型参数"""
        nn.init.xavier_uniform_(self.char_embeddings.weight)
        nn.init.xavier_uniform_(self.hidden2tag.weight)
        nn.init.constant_(self.hidden2tag.bias, 0)
    
    def forward(self, chars, lengths):
        """前向传播"""
        batch_size, seq_len = chars.size()
        
        # 字符嵌入
        embeddings = self.char_embeddings(chars)
        
        # 打包序列
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )
        
        # BiLSTM
        packed_lstm_out, _ = self.lstm(packed_embeddings)
        
        # 解包序列
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(packed_lstm_out, batch_first=True)
        
        # Dropout
        lstm_out = self.dropout(lstm_out)
        
        # 标签分数
        tag_scores = self.hidden2tag(lstm_out)
        
        return tag_scores
    
    def crf_loss(self, emissions, tags, lengths):
        """计算CRF损失"""
        batch_size, seq_len, num_tags = emissions.size()
        
        # 前向算法计算分配函数
        def forward_algorithm(emissions, transitions):
            batch_size, seq_len, num_tags = emissions.size()
            alpha = torch.full((batch_size, num_tags), -10000.0, device=emissions.device)
            alpha[:, 0] = emissions[:, 0, 0]  # 开始标签
            
            for i in range(1, seq_len):
                alpha_t = alpha.unsqueeze(2) + transitions.unsqueeze(0) + emissions[:, i].unsqueeze(1)
                alpha = torch.logsumexp(alpha_t, dim=1)
            
            return torch.logsumexp(alpha, dim=1)
        
        # 计算真实路径分数
        def score_sentence(emissions, tags, transitions):
            batch_size, seq_len = tags.size()
            score = torch.zeros(batch_size, device=emissions.device)
            
            # 第一个标签的发射分数
            score += emissions[range(batch_size), 0, tags[:, 0]]
            
            # 转移分数和发射分数
            for i in range(1, seq_len):
                score += transitions[tags[:, i-1], tags[:, i]]
                score += emissions[range(batch_size), i, tags[:, i]]
            
            return score
        
        # 计算前向分数
        forward_score = forward_algorithm(emissions, self.transitions)
        
        # 计算真实路径分数
        gold_score = score_sentence(emissions, tags, self.transitions)
        
        # 计算损失
        loss = forward_score - gold_score
        
        return loss.mean()

print("BiLSTM-CRF模型定义完成")

BiLSTM-CRF模型定义完成


In [15]:
# 构建字符词汇表
def build_vocab(char_sequences):
    char_counter = Counter()
    for chars in char_sequences:
        char_counter.update(chars)
    
    # 创建字符到索引的映射
    char2idx = {'<PAD>': 0, '<UNK>': 1}
    for char, count in char_counter.most_common():
        if count >= 1:  # 最小频次阈值
            char2idx[char] = len(char2idx)
    
    idx2char = {idx: char for char, idx in char2idx.items()}
    return char2idx, idx2char

# 构建标签词汇表
label2idx = {'<PAD>': 0, 'B': 1, 'M': 2, 'E': 3, 'S': 4}
idx2label = {idx: label for label, idx in label2idx.items()}

# 构建字符词汇表
char2idx, idx2char = build_vocab(train_chars)

print(f"字符词汇表大小: {len(char2idx)}")
print(f"标签数量: {len(label2idx)}")

# 创建数据集
train_dataset = ChineseSegmentationDataset(train_chars, train_labels, char2idx, label2idx)
test_dataset = ChineseSegmentationDataset(test_chars, test_labels, char2idx, label2idx)

# 创建数据加载器
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"训练批次数: {len(train_loader)}")
print(f"测试批次数: {len(test_loader)}")

字符词汇表大小: 5169
标签数量: 5
训练批次数: 2717
测试批次数: 125


In [16]:
# 模型超参数
vocab_size = len(char2idx)
embedding_dim = 128
hidden_dim = 256
num_tags = len(label2idx)
learning_rate = 0.001
num_epochs = 50

# 初始化模型
model = BiLSTMCRF(vocab_size, embedding_dim, hidden_dim, num_tags).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练函数
def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for chars, labels, lengths in tqdm(train_loader, desc="Training"):
        chars = chars.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # 前向传播
        emissions = model(chars, lengths)
        
        # 计算损失
        loss = model.crf_loss(emissions, labels, lengths)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

# 简化的评估函数（不使用CRF解码）
def evaluate_simple(model, test_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for chars, labels, lengths in test_loader:
            chars = chars.to(device)
            labels = labels.to(device)
            
            # 前向传播
            emissions = model(chars, lengths)
            
            # 计算损失
            loss = model.crf_loss(emissions, labels, lengths)
            total_loss += loss.item()
            
            # 简单预测（取最大值）
            predictions = torch.argmax(emissions, dim=-1)
            
            # 计算准确率（只考虑有效长度）
            for i, length in enumerate(lengths):
                pred = predictions[i, :length]
                true = labels[i, :length]
                correct += (pred == true).sum().item()
                total += length.item()
    
    accuracy = correct / total if total > 0 else 0
    avg_loss = total_loss / len(test_loader)
    
    return avg_loss, accuracy

# 训练模型
train_losses = []
val_losses = []
val_accuracies = []

print("开始训练BiLSTM模型...")

for epoch in range(num_epochs):
    # 训练
    train_loss = train_epoch(model, train_loader, optimizer, device)
    
    # 评估
    val_loss, val_acc = evaluate_simple(model, test_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Val Loss: {val_loss:.4f}')
        print(f'Val Accuracy: {val_acc:.4f}')
        print('-' * 50)

# 保存模型
torch.save({
    'model_state_dict': model.state_dict(),
    'char2idx': char2idx,
    'idx2char': idx2char,
    'label2idx': label2idx,
    'idx2label': idx2label,
    'vocab_size': vocab_size,
    'embedding_dim': embedding_dim,
    'hidden_dim': hidden_dim,
    'num_tags': num_tags
}, '../model/bilstm_segmentation.pth')

print("模型训练完成并已保存")

开始训练BiLSTM模型...


Training:   9%|▉         | 255/2717 [04:03<39:13,  1.05it/s]  


KeyboardInterrupt: 

In [ ]:
# 绘制训练曲线
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(val_accuracies, label='Val Accuracy', color='green')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(range(len(train_losses)), train_losses, 'b-', alpha=0.7, label='Train Loss')
plt.plot(range(len(val_losses)), val_losses, 'r-', alpha=0.7, label='Val Loss')
plt.fill_between(range(len(train_losses)), train_losses, alpha=0.3)
plt.fill_between(range(len(val_losses)), val_losses, alpha=0.3)
plt.title('Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('../result/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"最终验证准确率: {val_accuracies[-1]:.4f}")
print(f"最终训练损失: {train_losses[-1]:.4f}")
print(f"最终验证损失: {val_losses[-1]:.4f}")

In [ ]:
def bilstm_segment(model, text, char2idx, idx2label, device):
    """使用BiLSTM模型进行分词"""
    model.eval()
    
    # 转换为字符索引
    char_indices = [char2idx.get(char, char2idx['<UNK>']) for char in text]
    chars_tensor = torch.tensor([char_indices]).to(device)
    lengths = torch.tensor([len(char_indices)])
    
    with torch.no_grad():
        # 前向传播
        emissions = model(chars_tensor, lengths)
        
        # 简单预测（取最大值，实际应用中可以使用维特比算法）
        predictions = torch.argmax(emissions, dim=-1)
        pred_labels = [idx2label[pred.item()] for pred in predictions[0]]
    
    # 根据标签进行分词
    words = []
    current_word = ""
    
    for char, label in zip(text, pred_labels):
        if label == 'B':  # 词的开始
            if current_word:
                words.append(current_word)
            current_word = char
        elif label == 'M':  # 词的中间
            current_word += char
        elif label == 'E':  # 词的结束
            current_word += char
            words.append(current_word)
            current_word = ""
        elif label == 'S':  # 单字词
            if current_word:
                words.append(current_word)
            words.append(char)
            current_word = ""
    
    if current_word:
        words.append(current_word)
    
    return words

def compare_segmentation_methods(text):
    """比较不同分词方法的结果"""
    print(f"原文: {text}")
    print("-" * 50)
    
    # 最大匹配算法
    mm_result = mm_model.bidirectional_matching(text)
    print(f"最大匹配算法: {' / '.join(mm_result)}")
    
    # BiLSTM算法
    bilstm_result = bilstm_segment(model, text, char2idx, idx2label, device)
    print(f"BiLSTM算法: {' / '.join(bilstm_result)}")
    
    print("=" * 50)
    return mm_result, bilstm_result

# 测试不同的文本
test_texts = [
    "我爱北京天安门",
    "机器学习是人工智能的重要分支",
    "今天天气很好",
    "自然语言处理技术发展迅速",
    "北京大学是著名的高等学府"
]

print("分词结果比较:")
print("=" * 50)

results = {}
for text in test_texts:
    mm_result, bilstm_result = compare_segmentation_methods(text)
    results[text] = {
        'max_matching': mm_result,
        'bilstm': bilstm_result
    }

In [ ]:
# 保存分词结果
import json

# 保存详细结果
detailed_results = {
    'training_info': {
        'epochs': num_epochs,
        'final_train_loss': train_losses[-1],
        'final_val_loss': val_losses[-1],
        'final_val_accuracy': val_accuracies[-1],
        'vocab_size': vocab_size,
        'model_parameters': sum(p.numel() for p in model.parameters())
    },
    'segmentation_results': results
}

with open('../result/segmentation_results.json', 'w', encoding='utf-8') as f:
    json.dump(detailed_results, f, ensure_ascii=False, indent=2)

# 保存训练历史
training_history = {
    'train_losses': train_losses,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}

with open('../result/training_history.json', 'w', encoding='utf-8') as f:
    json.dump(training_history, f, indent=2)

# 创建结果总结表格
summary_data = []
for text, result in results.items():
    summary_data.append({
        '原文': text,
        '最大匹配': ' / '.join(result['max_matching']),
        'BiLSTM': ' / '.join(result['bilstm']),
        '最大匹配词数': len(result['max_matching']),
        'BiLSTM词数': len(result['bilstm'])
    })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('../result/segmentation_comparison.csv', index=False, encoding='utf-8')

print("实验总结:")
print("=" * 60)
print(f"1. 数据集: MSR中文分词数据集（示例）")
print(f"2. 训练样本数: {len(train_chars)}")
print(f"3. 测试样本数: {len(test_chars)}")
print(f"4. 字符词汇表大小: {vocab_size}")
print(f"5. 模型参数数量: {sum(p.numel() for p in model.parameters()):,}")
print(f"6. 最终验证准确率: {val_accuracies[-1]:.4f}")
print(f"7. 训练轮数: {num_epochs}")
print(f"8. 使用设备: {device}")
print("=" * 60)

print("\n分词方法比较:")
print(summary_df.to_string(index=False))

print(f"\n所有结果已保存到 ../result/ 目录")
print(f"模型已保存到 ../model/ 目录")
print("\n实验完成!")